In [1]:
import os
import urllib.request
import itertools
import time

import pandas as pd
from Bio import Entrez,SeqIO

## **Function**

In [2]:
def get_protein_sequences(uniprot_list):
    # This makes it so we match only the ENTRY field
    uniprot_list = ['id%3A'+id for id in uniprot_list]
    line = '+OR+'.join(uniprot_list)
    url = f'https://www.uniprot.org/uniprot/?query={line}&format=fasta'
    with urllib.request.urlopen(url) as f:
        fasta = f.read().decode('utf-8').strip()
    return fasta

def swissprot_paritial_search(seq):
    uniprot_list = list()
    interval = 500
    
    for index in range(0, len(seq), interval):
        search_seq = seq[index:(index+interval+1)]
        uniprot_list.append(get_protein_sequences(search_seq))
        print(index, end=" ")
        time.sleep(0.5)
        
    return uniprot_list

In [3]:
os.getcwd()

'/home/wmbio/WORK/gitworking/deeptrio/src'

In [4]:
CD_HIT_PHTH = "/home/wmbio/WORK/gitworking/deeptrio/data/CD-HIT_RESULT/"

## **Positive PPI**

### **BioGrid Dataset(Positive)**

In [5]:
'''
condition -> A AND B - swiss prot, Human, Physical interaction
'''
bioGrid = pd.read_csv("../data/Positive/BIOGRID-MV-Physical-4.4.206.mitab.txt", sep="\t")
condition = (bioGrid['Alt IDs Interactor A'].str.contains('swiss-prot', regex= True, na=False)) & \
            (bioGrid['Alt IDs Interactor B'].str.contains('swiss-prot', regex= True, na=False)) & \
            (bioGrid['Taxid Interactor A'].str.match('taxid:9606', na=False)) & \
            (bioGrid['Taxid Interactor B'].str.match('taxid:9606', na=False))

# uniprot 추출
bioGrid = bioGrid[condition].iloc[:, 2:4]
bioGrid.columns = ["Interactor_A", "Interactor_B"]

interA_bioGrid = bioGrid['Interactor_A'].str.extract('((?<=swiss-prot:)[a-zA-Z0-9]+)', expand=False).to_list()
interB_bioGrid = bioGrid['Interactor_B'].str.extract('((?<=swiss-prot:)[a-zA-Z0-9]+)', expand=False).to_list()

duplicated_pair_bioGrid = set(zip(interA_bioGrid, interB_bioGrid))
duplicated_pair_flat_bioGrid = list(set(itertools.chain(*duplicated_pair_bioGrid)))

print(len(duplicated_pair_flat_bioGrid))
print(len(duplicated_pair_bioGrid))

10450
96326


### **DIP(Positive)**

In [6]:
dip = pd.read_csv("../data/Positive/Hsapi20170205_DIP.txt", sep="\t")

condition = (dip['ID interactor A'].str.contains('uniprotkb:', regex=True, na=False)) & \
            (dip['ID interactor B'].str.contains('uniprotkb:', regex=True, na=False)) & \
            (dip['ID interactor A'] != dip['ID interactor B']) & \
            (dip['Taxid interactor A'].str.contains('Homo sapiens', na=False)) & \
            (dip['Taxid interactor B'].str.contains('Homo sapiens', na=False))

dip = dip[condition].iloc[:, :2].dropna()
dip.columns = ["Interactor_A", "Interactor_B"]

interA_dip = dip['Interactor_A'].str.extract('((?<=uniprotkb:)[a-zA-Z0-9]+)', expand=False).to_list()
interB_dip = dip['Interactor_B'].str.extract('((?<=uniprotkb:)[a-zA-Z0-9]+)', expand=False).to_list()

duplicated_pair_dip = set(zip(interA_dip, interB_dip))
duplicated_pair_flat_dip = list(set(itertools.chain(*duplicated_pair_dip)))

print(len(duplicated_pair_flat_dip))
print(len(duplicated_pair_dip))

3078
5009


### **uniprot search with BioGrid, DIP**

In [7]:
duplicated_pair_flat = duplicated_pair_flat_bioGrid + duplicated_pair_flat_dip

duplicated_pair = set()
duplicated_pair.update(duplicated_pair_bioGrid)
duplicated_pair.update(duplicated_pair_dip)
print(len(duplicated_pair))

protein_seq_raw = swissprot_paritial_search(duplicated_pair_flat)
protein_seq = "\n".join(protein_seq_raw)

99214
0 500 1000 1500 2000 2500 3000 3500 4000 4500 5000 5500 6000 6500 7000 7500 8000 8500 9000 9500 10000 10500 11000 11500 12000 12500 13000 13500 

### **HPRD(Positive)**

In [8]:
hrpd = pd.read_csv("../data/Positive/HPRD_Release9_062910/BINARY_PROTEIN_PROTEIN_INTERACTIONS_HPRD.txt", sep="\t", header=None)

hrpd = hrpd[hrpd[2] != hrpd[5]]
hrpd = hrpd[hrpd[6] != 'yeast 2-hybrid']

duplicated_pair_hrpd = set(zip(hrpd[2].to_list(), hrpd[5].to_list()))
duplicated_pair_flat_hrpd = list(set(itertools.chain(*duplicated_pair_hrpd)))

print(len(duplicated_pair_flat_hrpd))
print(len(duplicated_pair_hrpd))

7813
28057


* **RefSeq Search Entrez**

In [9]:
Entrez.email ="sempre813@gmail.com"

request = Entrez.efetch(db="protein", id=duplicated_pair_flat_hrpd, rettype="fasta")
seq_record = SeqIO.parse(request, "fasta")
seq_record = list(seq_record)
seq_record = [value for value in seq_record if 50 < len(''.join(value.seq)) < 1500]

iid = list(map(lambda x : '>' + x.id, seq_record))
seq = list(map(lambda x : "".join(x.seq), seq_record))

hrpd_zip = list(zip(iid, seq))

### **Run CD-HIT(BioGrid, DIP, HRPD)**

In [10]:
iid, seq, temp_seq = list(), list(), list()

for line in protein_seq.split("\n"):
    if line.startswith(">"):
        temp_seq = "".join(temp_seq)
        if len(temp_seq) <= 50 or len(temp_seq) >= 1500:
            temp_seq = list()
        else :
            iid.append(line)
            seq.append(temp_seq)
            temp_seq = list()
    else :
        temp_seq.append(line)
seq.append("".join(temp_seq))

# BioGrid+DIP, HRPD
id_seq_pair_raw = list(zip(iid, seq[1:])) + hrpd_zip

with open(CD_HIT_PHTH + 'biogrid_search.fasta', 'w') as f:
    for item in list(itertools.chain(*id_seq_pair_raw)):
        f.write("%s\n" % item)

* **bash CD-HIT**

In [11]:
!bash run_cd_hit.sh {CD_HIT_PHTH}biogrid_search.fasta {CD_HIT_PHTH}biogrid_search_cdhit.fasta

Program: CD-HIT, V4.8.1 (+OpenMP), Apr 07 2021, 10:57:21
Command: cd-hit -i
         /home/wmbio/WORK/gitworking/deeptrio/data/CD-HIT_RESULT/biogrid_search.fasta
         -o
         /home/wmbio/WORK/gitworking/deeptrio/data/CD-HIT_RESULT/biogrid_search_cdhit.fasta
         -n 2 -c 0.4 -T 15 -M 1000

Started: Wed Feb 23 08:43:42 2022
                            Output                              
----------------------------------------------------------------
total seq: 19975
longest and shortest : 1499 and 51
Total letters: 10481557
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 13M
Buffer          : 15 X 11M = 165M
Table           : 2 X 0M = 0M
Miscellaneous   : 0M
Total           : 179M

Table limit with the given memory limit:
Max number of representatives: 1068766
Max number of word counting entries: 102534677

# comparing sequences from          0  to       1175
.---------- new table with      547 representatives
# comparing sequences fro

In [12]:
cdhit_seq, iid, seq, temp_seq = list(), list(), list(), list()

with open(CD_HIT_PHTH + 'biogrid_search_cdhit.fasta', 'r') as f:
    for line in f:
        cdhit_seq.append(line.strip("\n"))

for index in range(len(cdhit_seq)):
    if index == 0:
        iid.append(cdhit_seq[index])  
    else :
        if cdhit_seq[index].startswith(">"):
            iid.append(cdhit_seq[index])
        else :
            seq.append(cdhit_seq[index])            
id_seq_pair = list(zip(iid, seq))

* **Set to DataFrame**

In [13]:
duplicated_pair.update(duplicated_pair_hrpd)

duplicated_pair_df = pd.DataFrame(duplicated_pair)
duplicated_pair_df.columns = ["Interactor_A", "Interactor_B"]

id_seq_pair_df = pd.DataFrame(id_seq_pair)

id_seq_pair_df.columns = ["id", "seq"]
len(id_seq_pair_df.dropna())

8533

In [14]:
refseq = id_seq_pair_df[id_seq_pair_df['id'].str.startswith(">NP")]

refseq['id'] = refseq['id'].str.replace(">","",1)

id_seq_pair_df['id'] = id_seq_pair_df['id'].str.extract('((?<=\\>sp\\|)[a-zA-Z0-9]+)', expand=False)

id_seq_pair_df = id_seq_pair_df.dropna().append(refseq, ignore_index=True)

/home/wmbio/anaconda3/envs/deeptrio-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


* **Check protein pairs in CD-Hit Database**

In [15]:
# pair df filter
duplicated_pair_df['Check_A'] = duplicated_pair_df['Interactor_A'].apply(lambda x : 1 if x in id_seq_pair_df.id.to_list() else 0)
duplicated_pair_df['Check_B'] = duplicated_pair_df['Interactor_B'].apply(lambda x : 1 if x in id_seq_pair_df.id.to_list() else 0)

duplicated_pair_df = duplicated_pair_df[(duplicated_pair_df.Check_A == 1) & (duplicated_pair_df.Check_B == 1)].iloc[:, 0:2]
duplicated_pair_df['label'] = [1 for _ in range(duplicated_pair_df.shape[0])]

In [16]:
# id seq pair filter
dup_pair_seq = set((list(duplicated_pair_df.Interactor_A.to_list()) + list(duplicated_pair_df.Interactor_B.to_list())))

id_seq_pair_df['Check_A'] = id_seq_pair_df['id'].apply(lambda x : 1 if x in dup_pair_seq else 0)
id_seq_pair_df['Check_B'] = id_seq_pair_df['id'].apply(lambda x : 1 if x in dup_pair_seq else 0)

id_seq_pair_df = id_seq_pair_df[(id_seq_pair_df.Check_A == 1) & (id_seq_pair_df.Check_B == 1)].iloc[:, 0:2]

In [17]:
# final
positivePair = duplicated_pair_df
positivePair_seq = id_seq_pair_df

## **Negative PPI**

### **Negatome database**

In [18]:
negatome = pd.read_csv("../data/Negative/16169070_neg.mitab", sep = "\t", header=None)

negatome_select = negatome.iloc[:, [0,1,14]]
negatome_select.columns = ['Interactor_A', 'Interactor_B', 'shortestPath']

# ID extraction
negatome_select['Interactor_A'] = negatome_select['Interactor_A'].str.extract('((?<=uniprotkb:)[a-zA-Z0-9]+)', expand=False)
negatome_select['Interactor_B'] = negatome_select['Interactor_B'].str.extract('((?<=uniprotkb:)[a-zA-Z0-9]+)', expand=False)
negatome_select['shortestPath'] = negatome_select['shortestPath'].str.extract('((?<=shortestPath:)[0-9]+)', expand=False)
negatome_select = negatome_select.dropna()
negatome_select = negatome_select.astype({'shortestPath': 'int'})

/home/wmbio/anaconda3/envs/deeptrio-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/wmbio/anaconda3/envs/deeptrio-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/wmbio/anaconda3/envs/deeptrio-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [19]:
negatome_filterd = negatome_select[negatome_select.shortestPath >= 7]

In [20]:
interA_negatome = negatome_filterd['Interactor_A'].to_list()
interB_negatome = negatome_filterd['Interactor_B'].to_list()

duplicated_pair_negatome = set(zip(interA_negatome, interB_negatome))
duplicated_pair_flat_negatome = list(set(itertools.chain(*duplicated_pair_negatome)))

print(len(duplicated_pair_flat_negatome))
print(len(duplicated_pair_negatome))

1566
73843


### **Negatome database2**

In [21]:
negatome2 = pd.read_csv("../data/Negative/combined_stringent.txt", sep = "\t", header=None)
negatome2.columns = ['Interactor_A', 'Interactor_B']

In [22]:
interA_negatome2 = negatome2['Interactor_A'].to_list()
interB_negatome2 = negatome2['Interactor_B'].to_list()

duplicated_pair_negatome2 = set(zip(interA_negatome2, interB_negatome2))
duplicated_pair_flat_negatome2 = list(set(itertools.chain(*duplicated_pair_negatome2)))

print(len(duplicated_pair_flat_negatome2))
print(len(duplicated_pair_negatome2))

3215
5808


* **search uniprot**

In [23]:
duplicated_pair_flat = duplicated_pair_flat_negatome + duplicated_pair_flat_negatome2

duplicated_pair = set()
duplicated_pair.update(duplicated_pair_negatome)
duplicated_pair.update(duplicated_pair_negatome2)
len(duplicated_pair)

protein_seq_raw = swissprot_paritial_search(duplicated_pair_flat)
protein_seq = "\n".join(protein_seq_raw)

0 500 1000 1500 2000 2500 3000 3500 4000 4500 

* **run CD-HIT**

In [24]:
iid, seq, temp_seq = list(), list(), list()

for line in protein_seq.split("\n"):
    if line.startswith(">"):
        temp_seq = "".join(temp_seq)
        if len(temp_seq) <= 50 or len(temp_seq) >= 1500:
            temp_seq = list()
        else :
            iid.append(line)
            seq.append(temp_seq)
            temp_seq = list()
    else :
        temp_seq.append(line)
seq.append("".join(temp_seq))

# BioGrid+DIP, HRPD
id_seq_pair_raw = list(zip(iid, seq[1:]))

with open(CD_HIT_PHTH + 'biogrid_search_negative.fasta', 'w') as f:
    for item in list(itertools.chain(*id_seq_pair_raw)):
        f.write("%s\n" % item)

In [25]:
!bash run_cd_hit.sh {CD_HIT_PHTH}biogrid_search_negative.fasta {CD_HIT_PHTH}biogrid_search_negative_cdhit.fasta

Program: CD-HIT, V4.8.1 (+OpenMP), Apr 07 2021, 10:57:21
Command: cd-hit -i
         /home/wmbio/WORK/gitworking/deeptrio/data/CD-HIT_RESULT/biogrid_search_negative.fasta
         -o
         /home/wmbio/WORK/gitworking/deeptrio/data/CD-HIT_RESULT/biogrid_search_negative_cdhit.fasta
         -n 2 -c 0.4 -T 15 -M 1000

Started: Wed Feb 23 08:50:01 2022
                            Output                              
----------------------------------------------------------------
total seq: 4303
longest and shortest : 1499 and 51
Total letters: 1887378
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 2M
Buffer          : 15 X 10M = 162M
Table           : 2 X 0M = 0M
Miscellaneous   : 0M
Total           : 165M

Table limit with the given memory limit:
Max number of representatives: 1224396
Max number of word counting entries: 104303178

# comparing sequences from          0  to        253
---------- new table with      192 representatives
# comparing

In [26]:
cdhit_seq, iid, seq, temp_seq = list(), list(), list(), list()

with open(CD_HIT_PHTH + 'biogrid_search_negative.fasta', 'r') as f:
    for line in f:
        cdhit_seq.append(line.strip("\n"))

for index in range(len(cdhit_seq)):
    if index == 0:
        iid.append(cdhit_seq[index])  
    else :
        if cdhit_seq[index].startswith(">"):
            iid.append(cdhit_seq[index])
        else :
            seq.append(cdhit_seq[index])            
id_seq_pair = list(zip(iid, seq))

In [27]:
duplicated_pair_df = pd.DataFrame(duplicated_pair)
duplicated_pair_df.columns = ["Interactor_A", "Interactor_B"]

id_seq_pair_df = pd.DataFrame(id_seq_pair)

id_seq_pair_df.columns = ["id", "seq"]
len(id_seq_pair_df.dropna())

4303

In [28]:
refseq = id_seq_pair_df[id_seq_pair_df['id'].str.startswith(">NP")]

refseq['id'] = refseq['id'].str.replace(">","",1)

id_seq_pair_df['id'] = id_seq_pair_df['id'].str.extract('((?<=\\>sp\\|)[a-zA-Z0-9]+)', expand=False)

id_seq_pair_df = id_seq_pair_df.dropna().append(refseq, ignore_index=True)

In [29]:
# pair df filter
duplicated_pair_df['Check_A'] = duplicated_pair_df['Interactor_A'].apply(lambda x : 1 if x in id_seq_pair_df.id.to_list() else 0)
duplicated_pair_df['Check_B'] = duplicated_pair_df['Interactor_B'].apply(lambda x : 1 if x in id_seq_pair_df.id.to_list() else 0)

duplicated_pair_df = duplicated_pair_df[(duplicated_pair_df.Check_A == 1) & (duplicated_pair_df.Check_B == 1)].iloc[:, 0:2]
duplicated_pair_df['label'] = [0 for _ in range(duplicated_pair_df.shape[0])]

In [30]:
# id seq pair filter
dup_pair_seq = set((list(duplicated_pair_df.Interactor_A.to_list()) + list(duplicated_pair_df.Interactor_B.to_list())))

id_seq_pair_df['Check_A'] = id_seq_pair_df['id'].apply(lambda x : 1 if x in dup_pair_seq else 0)
id_seq_pair_df['Check_B'] = id_seq_pair_df['id'].apply(lambda x : 1 if x in dup_pair_seq else 0)

id_seq_pair_df = id_seq_pair_df[(id_seq_pair_df.Check_A == 1) & (id_seq_pair_df.Check_B == 1)].iloc[:, 0:2]

In [31]:
# final
negativePair = duplicated_pair_df
negativePair_seq = id_seq_pair_df

In [32]:
negativePair_seq

,id,seq
0,Q6RFH5,MAAAAARWNHVWVGTETGILKGVNLQRKQAANFTAGGQPRREEAVS...
1,P17028,MSAQSVEEDSILIIPTPDEEEKILRVKLEEDPDGEEGSSIPWNHLP...
2,P47914,MAKSKNHTTHNQSRKWHRNGIKKPRSQRYESLKGVDPKFLRNMRFA...
3,P62328,MVTHSKFPAAGMSRPLDTSLRLKTFSSKSEYQLVVNAVRKLQESGF...
4,P49368,MMGHRPVLVLSQNTKRESGRKVQSGNINAAKTIADIIRTCLGPKSM...
...,...,...
3875,P48736,MELENYKQPVVLREDNCRRRRRMKPRSAAASLSSMELIPIEFVLPT...
3876,Q8N2W9,MAAELVEAKNMVMSFRVSDLQMLLGFVGRSKSGLKHELVTRALQLV...
3877,P15559,MVGRRALIVLAHSERTSFNYAMKEAAAAALKKKGWEVVESDLYAMN...
3878,Q3E7X9,MDNKTPVTLAKVIKVLGRTGSRGGVTQVRVEFLEDTSRTIVRNVKG...


## **Positive + Negative PPI**

In [33]:
TRAIN_PATH = "/home/wmbio/WORK/gitworking/deeptrio/data/Train_set/"

In [34]:
positivePair_set = set(zip(positivePair.Interactor_A.to_list(), positivePair.Interactor_B.to_list()))
negativePair_set = set(zip(negativePair.Interactor_A.to_list(), negativePair.Interactor_B.to_list()))

In [39]:
print(len(positivePair_set))
print(len(negativePair_set))

41287
66869


In [35]:
combine_dataset = positivePair.append(negativePair).drop_duplicates().sample(frac=1).reset_index(drop=True)

In [36]:
combine_dataset_seq = positivePair_seq.append(negativePair_seq).drop_duplicates().reset_index(drop=True)

* **write train_set**

In [37]:
combine_dataset.to_csv(TRAIN_PATH + "human_ppi_pair.tsv", sep="\t", index=False, header=False)

In [38]:
combine_dataset_seq.to_csv(TRAIN_PATH + "human_ppi_seq.tsv", sep="\t", index=False, header=False)